In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [2]:
df_model = pd.read_pickle('../data/pickle_emotion/df_model_en_.p')

In [3]:
df_model.head()

,count_words,count_mentions,count_hashtags,count_capital_words,count_excl_quest_marks,count_urls,count_emojis,sentiment,origin_text,clean_text
0,6,0,3,0,0,0,0,fear,Now #India is #afraid of #bad .,india afraid bad
1,8,0,2,0,1,0,0,anger,He's seriously so frustrating sometimes! (╯°□°）╯︵ ┻━┻ #ugh #raging,seriously frustrating sometimes ╯°□°）╯︵ ┻━┻ ugh raging
2,16,0,3,0,6,0,3,anger,How in the world did Nicole beat Paul?!?! #terrible #bb18 #BBFinale What was the jury thinking?? 😳😳😳,world nicole beat paul terrible bb bbfinale jury thinking 😳😳😳
3,11,0,3,0,0,0,0,anger,"Yeah, no. I don' mind tweetin' that twice. #snap #blm #likeThat",yeah no mind tweetin twice snap blm likethat
4,16,0,4,0,1,0,0,joy,What's the most important thing you need to accomplish today? #onlinemarketing #solopreneur #start up #success,important thing need accomplish today onlinemarketing solopreneur start success


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [5]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [6]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    

    features = FeatureUnion([('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [7]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.25, 0.5, 0.75)
}

In [8]:
mnb = MultinomialNB()

In [9]:
countvect = CountVectorizer()

In [10]:
# MultinomialNB
best_mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   45.7s finished


done in 49.046s

Best CV score: 0.575
Best parameters set:
	clf__alpha: 0.75
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.560


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.61      0.84      0.71       284
anticipation       0.29      0.16      0.21        95
     disgust       0.00      0.00      0.00        48
        fear       0.49      0.32      0.39        66
         joy       0.57      0.75      0.65       187
        love       0.00      0.00      0.00         1
    optimism       0.00      0.00      0.00        22
   pessimism       0.50      0.06      0.11        16
     sadness       0.00      0.00      0.00        22
    surprise       0.00      0.00      0.00         3

   micro avg       0.56      0.56      0.56       744
   macro avg       0.25      0.21      0.21       744
weighted avg       0.47      0.56 

In [11]:
tfidfvect = TfidfVectorizer()

In [12]:
best_mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   40.7s finished


done in 42.164s

Best CV score: 0.559
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 1)
Test score with best_estimator_: 0.543


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.55      0.90      0.68       284
anticipation       0.30      0.09      0.14        95
     disgust       0.00      0.00      0.00        48
        fear       0.43      0.15      0.22        66
         joy       0.58      0.69      0.63       187
        love       0.00      0.00      0.00         1
    optimism       0.00      0.00      0.00        22
   pessimism       1.00      0.06      0.12        16
     sadness       0.00      0.00      0.00        22
    surprise       0.00      0.00      0.00         3

   micro avg       0.54      0.54      0.54       744
   macro avg       0.29      0.19      0.18       744
weighted avg       0.45      0.54 

In [14]:
#max_df: 0.25 or maximum document frequency of 25%.
#min_df: 2 or the words need to appear in at least 2 tweets
#ngram_range: (1, 1)
#clf__alpha: 0.75
textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.25, min_df=2, ngram_range=(1,1)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', MultinomialNB(alpha=0.75))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)

In [16]:
df_model_pos = pd.read_pickle('../data/df_model_pos.p')
best_model.predict(df_model_pos).tolist()

['joy', 'joy', 'joy']

In [17]:
df_model_neg = pd.read_pickle('../data/df_model_neg.p')
best_model.predict(df_model_neg).tolist()

['anger', 'joy', 'anger', 'anger']